### Rest Api hits on https://dummyapi.io/

In [3]:
import requests
from functools import reduce
import psycopg2
import pandas as pd
from sshtunnel import SSHTunnelForwarder
from nose.tools import assert_true,assert_is_none
#get full user data
#api_url = "https://dummyapi.io/data/v1/user/60d0fe4f5311236168a109ca"
api_preffix = "https://dummyapi.io/data/v1/"
api_id = "6243344d20e61633b2e46ba8"
api_url = "https://dummyapi.io/data/v1/user?page=1&limit=50"
# api_url = "https://dummyapi.io/data/v1/user/60d0fe4f5311236168a10a01"
response = requests.get(api_url,  headers={"app-id": api_id, "Content-Type":"text"})
response_json = response.json()
list(response_json.keys())

['data', 'total', 'page', 'limit']

In [4]:
print("response sample :")
print(response_json)

response sample
{'data': [{'id': '60d0fe4f5311236168a109fd', 'title': 'mr', 'firstName': 'Dylan', 'lastName': 'Vasquez', 'picture': 'https://randomuser.me/api/portraits/med/men/66.jpg'}, {'id': '60d0fe4f5311236168a109fe', 'title': 'mr', 'firstName': 'AndrÃ©', 'lastName': 'Robert', 'picture': 'https://randomuser.me/api/portraits/med/men/9.jpg'}, {'id': '60d0fe4f5311236168a109ff', 'title': 'mrs', 'firstName': 'Josefina', 'lastName': 'Calvo', 'picture': 'https://randomuser.me/api/portraits/med/women/3.jpg'}, {'id': '60d0fe4f5311236168a10a00', 'title': 'mrs', 'firstName': 'Els', 'lastName': 'Ijsseldijk', 'picture': 'https://randomuser.me/api/portraits/med/women/75.jpg'}, {'id': '60d0fe4f5311236168a10a01', 'title': 'mr', 'firstName': 'Jesus', 'lastName': 'Riley', 'picture': 'https://randomuser.me/api/portraits/med/men/45.jpg'}, {'id': '60d0fe4f5311236168a10a02', 'title': 'mr', 'firstName': 'Cristobal', 'lastName': 'Soler', 'picture': 'https://randomuser.me/api/portraits/med/men/31.jpg'}, {'

In [5]:
# print(assert_true(response.ok))
# print(assert_is_none(response))

#### Retrieve all records over all pages

**retrieve the non-preview records across all page per data object (users, posts, comments))**

**ranges within RestAPI clause:**

**limit : [5-50]**

**pages : [0-999]**

In [6]:
def retreive_full_records(object_, api_preffix, api_id, page_upper_limit):
    #args: 
    # object_ : "user", "comment", "post"
    # api_preffix : https://dummyapi.io/data/v1/
    # api_id : 6243344d20e61633b2e46ba8
    # page_upper_limit : [0-999]
    records = []
    if object_ == "user":    
        location_dict = {}
        key_location = 0
        location_records = []    

    ### check of duplicated id using the frontier hash map
    id_frontier = {}

    def check_empty_response(response):    
        if "data" not in response.keys():
            return False    
        else:        
            if len(response["data"]) == 0:
                return False        
        return True

    def check_duplicated_ids (id_):
        nonlocal id_frontier
        try:
            if id_frontier[id_] == 1:
                return False
        except :        
            id_frontier[id_] = 1
        return True

    def get_detail_user_record (id_):
        ### get the full user information
        nonlocal api_preffix, object_, key_location
        if check_duplicated_ids(id_):        
            record = requests.get(api_preffix+object_+"/"+id_,  headers={"app-id":api_id, "Content-Type":"text"}).json()        
            print(record)
            if 'location' in record.keys():
                if len(record['location']) != 0:                
                    temp_location = record["location"]
                    info_concat = reduce(lambda a, b: a.strip() + b.strip(), list(record["location"].values()))
                    
                    if info_concat not in location_dict.keys():
                        location_dict[info_concat] = key_location
                        key_location += 1
                    record["location"] = key_location
                    temp_location["id"] = key_location
                    location_records.append(temp_location)
            return record
        return None

    def sanity_check_posts (record):   
        ### check for duplicated ids
        if not check_duplicated_ids(record['id']):
            return False    
        if "likes" not in record.keys():
            return False
        else:
            if record["likes"] < 0:
                return False
        if "owner" not in record.keys():
            return False
        else:
            if "id" not in record["owner"]:
                return False
            else:
                if record["owner"]["id"] == "":
                    return False
        return True

    def sanity_check_comments (record): 
        ### check for duplicated ids
        if not check_duplicated_ids(record['id']):
            return False    

        if "post" not in record.keys():
            return False
        else:
            if record["post"] == "":
                return False

        if "owner" not in record.keys():
            return False
        else:
            if "id" not in record["owner"]:
                return False
            else:
                if record["owner"]["id"] == "":
                    return False
        return True

    def fix_post_comment_records (record, page, check_post):
        if check_post:
            if sanity_check_posts(record):
                record["owner"] = record["owner"]["id"]
                record["page"]= page
                return record
        elif  sanity_check_comments(record):
            record["owner"] = record["owner"]["id"]
            record["page"]= page
            return record            
        return None

    for page in range(0,page_upper_limit):    
        api_url = api_preffix+object_+"?page="+str(page)+"&limit=50"
        response = requests.get(api_url,  headers={"app-id":api_id, "Content-Type":"text"})    
        assert_true(response.ok)
        response_json = response.json()    
        if check_empty_response(response_json):            
            #in case the object is user unnest the preview records, extract the location data
            if object_ == "user":             
                records += list(map(lambda x:  get_detail_user_record (x['id']), response_json["data"]))                
                #records += [get_detail_user_record (x['id']) for x in response_json["data"]]
                
            elif object_ == "post":
                records += list(map(lambda record : fix_post_comment_records(record, page, True), response_json["data"]))
                
                #records += [fix_post_comment_records(record, page, True) for record in response_json["data"]]
            elif object_ == "comment":
                records += list(map(lambda record : fix_post_comment_records(record, page, False), response_json["data"]))
                #records += [fix_post_comment_records(record, page, False) for record in response_json["data"]]
        else:
            print("Page number ",page)
            print(response_json)
        
    records = list(filter(lambda x:x is not None,records))        
    
    if object_ == "user":
        
        return pd.DataFrame(records), pd.DataFrame(location_records)
    
    elif object_ == "post":        
        pages = []
        look_up_posts_pages = {}
        for p in records:    
            key = p["id"]+str(p["page"])
            if key not in look_up_posts_pages:
                pages.append({'id':p["page"], "post_id":p["id"]})
                look_up_posts_pages [key] = 1
#             del p["page"]
        return pd.DataFrame(records), pd.DataFrame(pages)
        
    return pd.DataFrame(records)

_**Fetch user,location, post, comment records over a bunch of 999 pages.**_

_**In case you want the fetching to finish earlier reduce the page upper limit - no parralized implementation**_

In [7]:
print("Fetching Data....")
user, locations = retreive_full_records(object_ = "user", api_preffix = api_preffix, api_id = api_id, page_upper_limit = 999)
posts, pages = retreive_full_records(object_ = "post", api_preffix = api_preffix, api_id = api_id, page_upper_limit = 999)
comments = retreive_full_records(object_ = "comment", api_preffix = api_preffix, api_id = api_id, page_upper_limit = 999)

Fetching Data....
{'id': '60d0fe4f5311236168a109ca', 'title': 'ms', 'firstName': 'Sara', 'lastName': 'Andersen', 'picture': 'https://randomuser.me/api/portraits/women/58.jpg', 'gender': 'female', 'email': 'sara.andersen@example.com', 'dateOfBirth': '1996-04-30T19:26:49.610Z', 'phone': '92694011', 'location': {'street': '9614, SÃ¸ndermarksvej', 'city': 'Kongsvinger', 'state': 'Nordjylland', 'country': 'Denmark', 'timezone': '-9:00'}, 'registerDate': '2021-06-21T21:02:07.374Z', 'updatedDate': '2021-06-21T21:02:07.374Z'}
{'id': '60d0fe4f5311236168a109cb', 'title': 'miss', 'firstName': 'Edita', 'lastName': 'Vestering', 'picture': 'https://randomuser.me/api/portraits/med/women/89.jpg', 'gender': 'female', 'email': 'edita.vestering@example.com', 'dateOfBirth': '1956-04-15T00:10:35.555Z', 'phone': '(019)-646-0430', 'location': {'street': '1371, Dilledonk-Zuid', 'city': 'Den Bommel', 'state': 'Gelderland', 'country': 'Netherlands', 'timezone': '-5:00'}, 'registerDate': '2021-06-21T21:02:07.533

{'id': '60d0fe4f5311236168a109db', 'title': 'miss', 'firstName': 'Naomi', 'lastName': 'Rodrigues', 'picture': 'https://randomuser.me/api/portraits/med/women/39.jpg', 'gender': 'female', 'email': 'naomi.rodrigues@example.com', 'dateOfBirth': '1973-06-13T23:33:31.385Z', 'phone': '(40) 6623-4814', 'location': {'street': '9134, Rua Castro Alves ', 'city': 'Garanhuns', 'state': 'Roraima', 'country': 'Brazil', 'timezone': '+9:00'}, 'registerDate': '2021-06-21T21:02:10.280Z', 'updatedDate': '2021-06-21T21:02:10.280Z'}
{'id': '60d0fe4f5311236168a109dc', 'title': 'mr', 'firstName': 'Evan', 'lastName': 'Roux', 'picture': 'https://randomuser.me/api/portraits/med/men/59.jpg', 'gender': 'male', 'email': 'evan.roux@example.com', 'dateOfBirth': '1988-06-25T04:31:51.701Z', 'phone': '02-28-80-43-91', 'location': {'street': '892, Grande Rue', 'city': 'BesanÃ§on', 'state': 'Seine-et-Marne', 'country': 'France', 'timezone': '+9:00'}, 'registerDate': '2021-06-21T21:02:10.491Z', 'updatedDate': '2021-06-21T2

{'id': '60d0fe4f5311236168a109ec', 'title': 'mr', 'firstName': 'Lucas', 'lastName': 'Larsen', 'picture': 'https://randomuser.me/api/portraits/med/men/50.jpg', 'gender': 'male', 'email': 'lucas.larsen@example.com', 'dateOfBirth': '1987-04-04T15:06:47.176Z', 'phone': '54946998', 'location': {'street': '4187, Nattergalevej', 'city': 'St.MerlÃ¸se', 'state': 'Syddanmark', 'country': 'Denmark', 'timezone': '-3:00'}, 'registerDate': '2021-06-21T21:02:13.276Z', 'updatedDate': '2021-06-21T21:02:13.276Z'}
{'id': '60d0fe4f5311236168a109ed', 'title': 'miss', 'firstName': 'Kayla', 'lastName': 'Bredesen', 'picture': 'https://randomuser.me/api/portraits/med/women/13.jpg', 'gender': 'female', 'email': 'kayla.bredesen@example.com', 'dateOfBirth': '1958-08-20T08:43:07.057Z', 'phone': '80844280', 'location': {'street': '5225, Hansemyrveien', 'city': 'Vingrom', 'state': 'Finnmark - FinnmÃ¡rku', 'country': 'Norway', 'timezone': '+11:00'}, 'registerDate': '2021-06-21T21:02:13.446Z', 'updatedDate': '2021-06-

{'id': '60d0fe4f5311236168a109fe', 'title': 'mr', 'firstName': 'AndrÃ©', 'lastName': 'Robert', 'picture': 'https://randomuser.me/api/portraits/med/men/9.jpg', 'gender': 'male', 'email': 'andre.robert@example.com', 'dateOfBirth': '1960-04-08T15:04:24.730Z', 'phone': '077 689 97 19', 'location': {'street': '5684, Rue Denfert-Rochereau', 'city': 'Unterlangenegg', 'state': 'Fribourg', 'country': 'Switzerland', 'timezone': '-12:00'}, 'registerDate': '2021-06-21T21:02:16.329Z', 'updatedDate': '2021-06-21T21:02:16.329Z'}
{'id': '60d0fe4f5311236168a109ff', 'title': 'mrs', 'firstName': 'Josefina', 'lastName': 'Calvo', 'picture': 'https://randomuser.me/api/portraits/med/women/3.jpg', 'gender': 'female', 'email': 'josefina.calvo@example.com', 'dateOfBirth': '1982-01-27T16:01:51.717Z', 'phone': '976-538-478', 'location': {'street': '8941, Calle de Argumosa', 'city': 'Orihuela', 'state': 'Ceuta', 'country': 'Spain', 'timezone': '0:00'}, 'registerDate': '2021-06-21T21:02:16.491Z', 'updatedDate': '20

{'id': '60d0fe4f5311236168a10a0f', 'title': 'mr', 'firstName': 'Kaya', 'lastName': 'Basoglu', 'picture': 'https://randomuser.me/api/portraits/med/men/59.jpg', 'gender': 'male', 'email': 'kaya.basoglu@example.com', 'dateOfBirth': '1952-07-19T21:48:06.254Z', 'phone': '(686)-297-4736', 'location': {'street': '2574, Necatibey Cd', 'city': 'Amasya', 'state': 'BalÄ±kesir', 'country': 'Turkey', 'timezone': '+8:00'}, 'registerDate': '2021-06-21T21:02:18.992Z', 'updatedDate': '2021-06-21T21:02:18.992Z'}
{'id': '60d0fe4f5311236168a10a10', 'title': 'mr', 'firstName': 'Nenad', 'lastName': 'Leroy', 'picture': 'https://randomuser.me/api/portraits/med/men/62.jpg', 'gender': 'male', 'email': 'nenad.leroy@example.com', 'dateOfBirth': '1949-02-02T00:32:14.911Z', 'phone': '078 789 60 53', 'location': {'street': '6975, Rue AndrÃ©-Gide', 'city': 'Arni (Ag)', 'state': 'Basel-Landschaft', 'country': 'Switzerland', 'timezone': '-3:00'}, 'registerDate': '2021-06-21T21:02:19.188Z', 'updatedDate': '2021-06-21T21

JSONDecodeError: [Errno Expecting value] Internal Error: 0

In [ ]:
print("Data Review")
display(users)

In [ ]:
print("Locations Frame")
display(locations)

In [ ]:
print("Posts Frame ....")
display(posts)

In [ ]:
print("Comments Frame ....")
display(comments)

In [ ]:
import sys

### Connect to the database

In [12]:
print("DataBase (Postgres) Connection ..")

DataBase (Postgres) Connection ..


In [13]:
ssh_tunnel= SSHTunnelForwarder(
         ('snf-880201.vm.okeanos.grnet.gr', 22),
         ssh_password="pantelis123",
         ssh_username="root",
         remote_bind_address=('172.20.0.2', 5432))

ssh_tunnel.start()

ssh_tunnel.local_bind_port

connection = psycopg2.connect(dbname='dummyapidb'
                              , user="postgres"
                              , password="postgres"
                              , host='127.0.0.1'
                              , port=ssh_tunnel.local_bind_port)#127.0.0.1

#connection = psycopg2.connect(dbname='dummyapidb',
#                              user = "postgres",
#                              password = "postgres",
#                              host = "localhost",
#                              port = "5432")

cursor = connection.cursor()

2022-04-04 16:50:32,327| ERROR   | Could not establish connection from local ('127.0.0.1', 51100) to remote ('172.18.0.2', 5432) side of the tunnel: open new channel ssh error: Timeout opening channel.


OperationalError: connection to server at "127.0.0.1", port 51100 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


### Tables Initialization

In [ ]:
print("Staging Tables Initialization ..")

In [ ]:
create_users_tabel = ("""
DROP TABLE IF EXISTS UsersStage cascade;
CREATE TABLE IF NOT EXISTS UsersStage (
  id varchar(255) PRIMARY KEY,
  title varchar(255),
  firstName varchar(255),
  lastName varchar(255),
  picture varchar(255),
  gender varchar(255),
  email varchar(255),
  dateOfBirth date DEFAULT NULL,
  phone varchar(255),
  location_id int,
  registerDate date DEFAULT NULL,
  updatedDate date DEFAULT NULL
);
""")

create_locations_tabel = ("""
DROP TABLE IF EXISTS LocationsStage cascade;
CREATE TABLE IF NOT EXISTS LocationsStage (
  id INT PRIMARY KEY,
  street varchar(255),
  city varchar(255),
  state varchar(255),
  country varchar(255),
  timezone varchar(255)
);
""")

create_posts_tabel = ("""
DROP TABLE IF EXISTS Posts cascade;
CREATE TABLE IF NOT EXISTS Posts (
  id varchar(255),
  image varchar(255),
  likes int,
  tags varchar(255) ARRAY,
  text varchar(255),
  publishDate date DEFAULT NULL,
  user_id varchar(255),
  page_id INT
);
""")

create_comments_tabel = ("""
DROP TABLE IF EXISTS Comments cascade;
CREATE TABLE IF NOT EXISTS Comments (
  id varchar(255),
  message varchar(255),
  user_id varchar(255),
  post_id varchar(255),
  publishDate date DEFAULT NULL,
  page_id INT
);
""")

# """
# Fact Table : pages related to posts
# """
# create_pages_tabel = ("""
# DROP TABLE IF EXISTS Pages cascade;
# CREATE TABLE IF NOT EXISTS Pages (
#   id INT PRIMARY KEY,
#   post_id varchar(255)
# );
# """)

set_data_null_raw_data = (
    """
    UPDATE RawData rd
    SET PublishDate = CASE WHEN rd.PublishDate =  '-INFINITY' THEN NULL ELSE rd.PublishDate END;
    """
    )

### Create the Staging Dimension Data Tables (Users, Locations) & Fact tables (Posts, Comments, Pages)
cursor.execute(create_users_tabel)
cursor.execute(create_locations_tabel)
cursor.execute(create_comments_tabel)
cursor.execute(create_posts_tabel)
# cursor.execute(create_pages_tabel)
#connection.commit()

In [ ]:
# cursor.close()
# connection.close()
# posts
# connection.commit()

### Insert the Data Records (Staging Phase)

In [ ]:
print("Data ingestion (staging) phase")

In [ ]:
def ingest_raw_user_data (raw_data, cursor, connection):
    insert_row_RawData = (
    """
    INSERT INTO UsersStage (    
    id,
    title,
    firstName,
    lastName,
    picture,
    gender,
    email,
    dateOfBirth,
    phone,
    location_id,
    registerDate,
    updatedDate
    ) VALUES (%s, %s, %s, %s, %s,%s,%s, %s, %s, %s, %s,%s)
    ON CONFLICT ON CONSTRAINT usersstage_pkey DO NOTHING;
    """)
    for index, row in raw_data.iterrows():                
        row_ = tuple(row.to_list())
        cursor.execute(insert_row_RawData, row_)
        
        
def ingest_raw_location_data (raw_data, cursor, connection):
    insert_row_RawData = (
    """
    INSERT INTO LocationsStage (    
    street,
    city ,
    state,
    country,
    timezone,
    id
    ) VALUES (%s, %s, %s, %s, %s,%s)
    ON CONFLICT ON CONSTRAINT locationsstage_pkey DO NOTHING;
    """)
    for index, row in raw_data.iterrows():                
        row_ = tuple(row.to_list())        
        cursor.execute(insert_row_RawData, row_)
        
def ingest_raw_posts_data (raw_data, cursor, connection):
    insert_row_RawData = (
    """
    ALTER TABLE Posts
    DROP CONSTRAINT IF EXISTS UniqSignPosts;
    ALTER TABLE Posts
    ADD CONSTRAINT UniqSignPosts UNIQUE (id );
    INSERT INTO Posts (    
    id ,
    image,
    likes,
    tags ,
    text ,
    publishDate ,
    user_id,
    page_id
    ) VALUES (%s, %s, %s, %s, %s,%s,%s,%s)
    ON CONFLICT ON CONSTRAINT UniqSignPosts DO NOTHING;
    """)
    for index, row in raw_data.iterrows():                
        row_ = tuple(row.to_list())
        cursor.execute(insert_row_RawData, row_)
        
def ingest_raw_comments_data (raw_data, cursor, connection):
    insert_row_RawData = (
    """
    ALTER TABLE Comments
    DROP CONSTRAINT IF EXISTS UniqSignComments;
    ALTER TABLE Comments
    ADD CONSTRAINT UniqSignComments UNIQUE (id );
    INSERT INTO Comments (    
    id,
    message,
    user_id,
    post_id,
    publishDate,
    page_id
    ) VALUES (%s, %s, %s, %s, %s,%s)
    ON CONFLICT ON CONSTRAINT UniqSignComments DO NOTHING;
    """)
    for index, row in raw_data.iterrows():                
        row_ = tuple(row.to_list())
        cursor.execute(insert_row_RawData, row_)
        
def ingest_raw_pages_data (raw_data, cursor, connection):
    insert_row_RawData = (
    """
    INSERT INTO Pages (    
    id,
    post_id
    ) VALUES (%s, %s)
    """)
    for index, row in raw_data.iterrows():                
        row_ = tuple(row.to_list())
        cursor.execute(insert_row_RawData, row_)
        
ingest_raw_user_data (user, cursor, connection)    
ingest_raw_posts_data (posts, cursor, connection)
ingest_raw_location_data (locations, cursor, connection)
ingest_raw_comments_data (comments, cursor, connection)
# ingest_raw_pages_data (pages, cursor, connection)
connection.commit()

### Define the DW Dimensional Tables (Eliminate Duplication)

In [ ]:
print("DW Dimensional Tables Definition ..")

In [ ]:
create_users_tabel = ("""
DROP TABLE IF EXISTS Users cascade;
CREATE TABLE IF NOT EXISTS Users (
  id bigint GENERATED BY DEFAULT AS IDENTITY NOT NULL primary key, 
  old_id varchar(255),
  title varchar(255),
  firstName varchar(255),
  lastName varchar(255),
  picture varchar(255),
  gender varchar(255),
  email varchar(255),
  dateOfBirth date DEFAULT NULL,
  phone varchar(255),
  location_id int,
  registerDate date DEFAULT NULL,
  updatedDate date DEFAULT NULL
);
""")

create_locations_tabel = ("""
DROP TABLE IF EXISTS Locations cascade;
CREATE TABLE IF NOT EXISTS Locations (
  id bigint GENERATED BY DEFAULT AS IDENTITY NOT NULL primary key,
  old_id INT,
  street varchar(255),
  city varchar(255),
  state varchar(255),
  country varchar(255),
  timezone varchar(255)
);
""")

cursor.execute(create_users_tabel)
cursor.execute(create_locations_tabel)

### Populate DW Data Tables
**Further sanity check for duplication by adding the respective constraints**

In [ ]:
print("Populate DW Data Tables..")

In [ ]:
config_user =("""

ALTER TABLE Users
    ADD CONSTRAINT UniqSignUsers UNIQUE (title, firstName, lastName, gender, email,dateOfBirth );

INSERT INTO Users(
      old_id,
      title ,
      firstName ,
      lastName,
      picture ,
      gender ,
      email,
      dateOfBirth,
      phone ,
      location_id,
      registerDate ,
      updatedDate
    ) 
SELECT   
  id ,
  title ,
  firstName ,
  lastName ,
  picture ,
  gender ,
  email ,
  dateOfBirth ,
  phone ,
  location_id ,
  registerDate ,
  updatedDate 
FROM UsersStage
ON CONFLICT ON CONSTRAINT UniqSignUsers DO NOTHING;
""")
    
config_location = ("""

ALTER TABLE Locations
    ADD CONSTRAINT UniqSignLocation UNIQUE (street,city, state,country);

INSERT INTO Locations(
    old_id ,
    street,
    city,
    state,
    country,
    timezone
    ) 
SELECT id, 
       street
       ,city
       ,state
       ,country
       ,timezone
FROM LocationsStage
ON CONFLICT ON CONSTRAINT UniqSignLocation DO NOTHING;

""")

cursor.execute(config_user)
cursor.execute(config_location)
connection.commit()

### Update new Dimensional FK Ids on Fact Tables

In [ ]:
print("Update Dimensional FK ids on Fact Tables")

In [ ]:
update_posts_user_id_FK = ("""
        UPDATE posts
        SET user_id = users.id
        FROM users
        WHERE users.old_id = posts.user_id;
        ALTER TABLE posts
        ALTER COLUMN user_id TYPE bigint USING user_id::bigint;
        """)

update_comments_user_id_FK = ("""
        UPDATE comments
        SET user_id = users.id
        FROM users
        WHERE users.old_id = comments.user_id;
        ALTER TABLE comments
        ALTER COLUMN user_id TYPE bigint USING user_id::bigint;
        """)
 
cursor.execute(update_posts_user_id_FK)
cursor.execute(update_comments_user_id_FK)
connection.commit()

### Set PK on Posts & Comments

In [ ]:
print("Set PK on Facts Tables")

In [ ]:
set_PK_posts = ("""
        ALTER TABLE posts
        ADD CONSTRAINT posts_pk PRIMARY KEY (id);
        """)
set_PK_comments = ("""
        ALTER TABLE comments
        ADD CONSTRAINT comments_pk PRIMARY KEY (id);
        """)

cursor.execute(set_PK_posts)
cursor.execute(set_PK_comments)
connection.commit()

### Table Intercorrelation Definitions

In [ ]:
print("Define correlations amognst tables")

In [ ]:
users_locations_many_to_one = ("""
ALTER TABLE Users ADD FOREIGN KEY (location_id) REFERENCES  Locations(id);
""")

posts_users_many_to_one = ("""
ALTER TABLE Posts ADD FOREIGN KEY (user_id) REFERENCES  Users(id);
""")

pages_posts_one_to_many = ("""
ALTER TABLE Pages ADD FOREIGN KEY (post_id) REFERENCES  Posts(id);
""")

# comments_pages_many_to_one = ("""
# ALTER TABLE Comments ADD FOREIGN KEY (page_id) REFERENCES  Pages(id);
# """)

comments_owner_many_to_one = ("""
ALTER TABLE Comments ADD FOREIGN KEY (user_id) REFERENCES  Users(id);
""")

cursor.execute(users_locations_many_to_one)
cursor.execute(posts_users_many_to_one)
# cursor.execute(pages_posts_one_to_many)
# cursor.execute(comments_pages_many_to_one)
cursor.execute(comments_owner_many_to_one)
connection.commit()

### Data Modeling Finished : Perform the Queries

#### The final data model looks like the above :

<img src="DummyAPIDB Schema.png" width=1600 height=1600 />

In [ ]:
print("Perfoming Respective Queries")

In [ ]:
# --  How many new users are added daily?

cursor.execute("""
select round(avg(new_users_added_daily),2) as avg_user_added from (
select count(distinct(id)) as new_users_added_daily from users
group by registerdate) as temp;""")
print("How many new users are added daily?")
print(cursor.fetchall())

# -- Which cities have the most activity, in terms of posts per day?

cursor.execute("""with posts_country as(
select posts.id as post_ids, locations.city as city from posts
left join users on users.id = posts.user_id
left join locations on locations.id = users.location_id)
select city, count(post_ids) as post_volumes from posts_country
group by city
order by post_volumes desc limit 10;""")
print("Which cities have the most activity, in terms of posts per day?")
print(cursor.fetchall())

# -- Which tags are most frequently encountered, across user posts?

cursor.execute("""select tags, count(*) as tags_volume_across_posts from posts
group by tags
having count(*) > 1
order by tags_volume_across_posts desc limit 10;""")
print("Which tags are most frequently encountered, across user posts?")
print(cursor.fetchall())

# -- What is the median comment/post ratio amongst the users with the most posts

cursor.execute("""
DROP FUNCTION IF EXISTS _final_median(INT);

CREATE OR REPLACE FUNCTION _final_median(numeric[])
   RETURNS numeric AS
$$
   SELECT AVG(val)
   FROM (
     SELECT val
     FROM unnest($1) val
     ORDER BY 1
     LIMIT  2 - MOD(array_upper($1, 1), 2)
     OFFSET CEIL(array_upper($1, 1) / 2.0) - 1
   ) sub;
$$
LANGUAGE 'sql' IMMUTABLE;

DROP AGGREGATE IF EXISTS median(numeric);

CREATE AGGREGATE median(numeric) (
  SFUNC=array_append,
  STYPE=numeric[],
  FINALFUNC=_final_median,
  INITCOND='{}'
);

with user_comments_post as (
select users.*,posts.id as posts_id,comments.id as comments_id from users
left join comments on comments.user_id = users.id
left join posts on posts.user_id = users.id)
select round(median(comment_post_ratio),2) from (
select id, count(distinct(comments_id))/count(distinct(posts_id)) as comment_post_ratio   from user_comments_post
group by id
order by count(posts_id) desc limit 10) as comment_post_ratio_;""")
print("What is the median comment/post ratio amongst the users with the most posts?")
print(cursor.fetchall())
# -- What is the average time between registration and first comment?

cursor.execute("""with user_comments as (
select users.id as users_id, users.registerdate as users_registerdate, comments.publishdate as comment_publishdate, comments.id as comments_id from users
left join comments on comments.user_id = users.id)
select round(avg(diff_register_first_comment_publish_date),2) from (
select users_id,min(comment_publishdate) -  min(users_registerdate) as diff_register_first_comment_publish_date from user_comments
group by users_id) as user_comments_results""")
print("What is the average time between registration and first comment?")
print(cursor.fetchall())
print("#### the last query result is peculiar due to data source incosystency in defining the users registration dates !")

In [ ]:
cursor.close()
connection.close()

2022-04-04 16:52:30,470| ERROR   | Secsh channel 0 open FAILED: Connection timed out: Connect failed


#### the last query result is peculiar due to data source incosystency in defining the users registration dates !

### Apache - Airflow usability
1. Monitoring Cron jobs
2. transferring data from one place to other.
3. Automating your DevOps operations.
4. Periodically fetching data from websites and update the database for your awesome price comparison system.
5. Data processing for recommendation based systems.
6. Machine Learning Pipelines.

**Despite the fact that the current pipeline was performed from the scratch and in manuall manner**

*One of the possible tools that could be leveraged for the unit testing about data validity as well as the data transformation to the final DW modeling could be open source-oriented packages like dbt*

*The capabilities that dbt offers like snashots (roll-back operation to previous stages and monitoring of changes) 
could facilitate the ETL process especially in the production phases*

*On the top of DBT, airflow could be plugeed in for monitoring the DBT unit test and triggered alerts whenever it's neccecary, making a full end-to-end monitorable ETL pipeline*

### Airflow Operators 
* BashOperator - executes a bash command
* PythonOperator - calls an arbitrary Python function
* EmailOperator - sends an email
* SimpleHttpOperator - sends an HTTP request
* MySqlOperator, SqliteOperator, PostgresOperator, MsSqlOperator, OracleOperator, JdbcOperator, etc. - executes a SQL command
* Sensor - waits for a certain time, file, database row, S3 key, etc…